In [2]:
# Dependencies
import plotly.graph_objs as go
from shapely.geometry import shape
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json

### Migration flows by Region

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [9]:
# Create df for migration into England and Wales
mig_rgn_in_df = pd.read_csv("./Resources/MIG004EW_RGN_IN.csv")
# print(mig_rgn_in_df.head())

# Create df for emigration from England and Wales
mig_rgn_out_df = pd.read_csv("./Resources/MIG004EW_RGN_OUT.csv")
# mig_rgn_out_df.head()

In [10]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Migration Region (inflow) (6 categories) code', 'Migration Region (inflow) (6 categories) label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)

#Save df to csv file
mig_rgn_in.to_csv("./Resources/local_authority_in.csv", index=False)


# Count local regions out flow
mig_rgn_out = mig_rgn_out_df.groupby(['Migration Region (outflow) (3 categories) code', 'Migration Region (outflow) (3 categories) label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)


#Save df to csv file
mig_rgn_out.to_csv("./Resources/local_authority_out.csv", index=False)

print(mig_rgn_out)

   Migration Region (outflow) (3 categories) code  \
0                                              -8   
1                                               1   
2                                               2   

     Migration Region (outflow) (3 categories) label     Count  
0                                     Does not apply  57800607  
1  Outflow: moved out of the area, but within the...   1091937  
2  Outflow: moved out of the associated area, but...     97709  


### Merge of Net Migration information by region

In [18]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Regions code','Regions label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)
mig_rgn_in

mig_rgn_out = mig_rgn_out_df.groupby(['Migrant region one year ago code', 'Migrant region one year ago label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)
# mig_rgn_out

# Merge DataFrames on the common column
region_df = pd.merge(mig_rgn_in, mig_rgn_out, left_on='Regions code', right_on='Migrant region one year ago code', how='left')

# Fill NaN values with 0
region_df[['Count_x', 'Count_y']] = region_df[['Count_x', 'Count_y']].fillna(0)

# Calculate net migration
region_df['Net Migration'] = region_df['Count_x'] - region_df['Count_y']

# Select relevant columns
region_net_migration = region_df[['Regions code', 'Regions label', 'Net Migration']]
region_net_migration



,Regions code,Regions label,Net Migration
0,E12000008,South East,8246470
1,E12000007,London,7520262
2,E12000002,North West,6688909
3,E12000006,East of England,5687069
4,E12000005,West Midlands,5368354
5,E12000009,South West,5067886
6,E12000003,Yorkshire and The Humber,4918006
7,E12000004,East Midlands,4373113
8,W92000004,Wales,2818571
9,E12000001,North East,2388495
